

# Mounting

 **First  We mount the drive to goolgle drive where we have all the dev set and test set **

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
 os.chdir('/content/drive/My Drive/CA684_Assignment')


# Import Packages

In [4]:
!pip install pyprind

In [5]:
import pandas as pd

from tensorflow.python.keras import Sequential
from tensorflow.python.keras import layers
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.text import Tokenizer

import numpy as np
from string import punctuation
import pyprind
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn import neighbors

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# DATA SET LOAD

**1.Importing the caption into data frame(Training or Dev set)**




---



In [9]:
caption_train=pd.read_csv('Dev-set/Captions/dev-set_video-captions.txt',sep='\t',names=('video','caption'))
print(caption_train['caption'])

0                      blonde-woman-is-massaged-tilt-down
1       roulette-table-spinning-with-ball-in-closeup-shot
2                                           khr-gangsters
3                    medical-helicopter-hovers-at-airport
4                    couple-relaxing-on-picnic-crane-shot
                              ...                        
5995          beautiful-young-woman-in-front-of-fountains
5996    focus-pull-from-molting-penguin-to-penguin-col...
5997             students-walking-in-university-of-mexico
5998                         beautiful-black-woman-at-spa
5999    beautiful-modern-brunette-woman-using-tablet-c...
Name: caption, Length: 6000, dtype: object


**2.Importing the Ground truth in data frame(Training or Dev)**

---



In [10]:
grndtrth_train=pd.read_csv('Dev-set/Ground-truth/ground-truth.csv',sep=',',usecols=[0,1,3])
print(grndtrth_train)

               video  short-term_memorability  long-term_memorability
0        video3.webm                    0.924                   0.846
1        video4.webm                    0.923                   0.667
2        video6.webm                    0.863                   0.700
3        video8.webm                    0.922                   0.818
4       video10.webm                    0.950                   0.900
...              ...                      ...                     ...
5995  video7488.webm                    0.921                   1.000
5996  video7489.webm                    0.909                   0.839
5997  video7491.webm                    0.713                   0.818
5998  video7492.webm                    0.954                   1.000
5999  video7493.webm                    0.953                   0.615

[6000 rows x 3 columns]


**3.Importing the test set for video caption in data frame(Test Set)**

In [11]:
caption_test=pd.read_csv('Test-set/Captions_test/test-set-1_video-captions.txt',sep='\t',names=('video','caption'))
print(caption_test)

                video                                            caption
0      video7494.webm     green-jeep-struggling-to-drive-over-huge-rocks
1      video7495.webm  hiking-woman-tourist-is-walking-forward-in-mou...
2      video7496.webm  close-up-of-african-american-doctors-hands-usi...
3      video7497.webm  slow-motion-of-a-man-using-treadmill-in-the-gy...
4      video7498.webm       slow-motion-of-photographer-in-national-park
...               ...                                                ...
1995  video10004.webm  astronaut-in-outer-space-against-the-backdrop-...
1996  video10005.webm  young-women-lying-on-sunbed-and-applying-sun-c...
1997  video10006.webm  doctor-talking-to-patient-using-a-tablet-to-ex...
1998  video10007.webm  businessman-sitting-on-the-beach-on-inflatable...
1999  video10008.webm  woman-eating-ice-cream-and-sitting-in-the-stre...

[2000 rows x 2 columns]


**4.Importing the test set for ground truth in data frame(Test Set)**

In [12]:
grndtrth_test=pd.read_csv('Test-set/Ground-truth_test/ground_truth_template.csv')
print(grndtrth_test)

      video  ...  nb_long-term_annotations
0      7494  ...                        12
1      7495  ...                        10
2      7496  ...                        13
3      7497  ...                        10
4      7498  ...                        10
...     ...  ...                       ...
1995  10004  ...                        17
1996  10005  ...                         9
1997  10006  ...                        12
1998  10007  ...                        12
1999  10008  ...                        10

[2000 rows x 5 columns]


**5.Combining train and test of caption for pre-processing together**

In [13]:
combined_caption=pd.concat([caption_train,caption_test],axis=0,ignore_index=True)
combined_caption.shape

(8000, 2)

# Pre-Processing Caption

**1.Defining a function for pre processing(removal of noise eg.-,@etc**

---



In [14]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text
 

combined_caption['caption'] = combined_caption['caption'].apply(lambda x:pre_process(x))
 
#show the second 'text' just for fun
combined_caption['caption'][1]

'roulette table spinning with ball in closeup shot'

**2.Using Stemmer,TF-idfvectorize,removing stopwords from Caption**

---



In [15]:
# init stemmer
porter_stemmer=PorterStemmer()


# # Split the sentences to lists of words.
combined_caption['caption'] = combined_caption['caption'].str.split()

combined_caption['caption'] = combined_caption['caption'].apply(lambda x: [porter_stemmer.stem(y) for y in x]) # Stem every word.


combined_caption['caption']

 
 

0                  [blond, woman, is, massag, tilt, down]
1       [roulett, tabl, spin, with, ball, in, closeup,...
2                                         [khr, gangster]
3                   [medic, helicopt, hover, at, airport]
4                 [coupl, relax, on, picnic, crane, shot]
                              ...                        
7995    [astronaut, in, outer, space, against, the, ba...
7996    [young, women, lie, on, sunb, and, appli, sun,...
7997    [doctor, talk, to, patient, use, a, tablet, to...
7998    [businessman, sit, on, the, beach, on, inflat,...
7999    [woman, eat, ice, cream, and, sit, in, the, st...
Name: caption, Length: 8000, dtype: object

In [16]:
stop = stopwords.words('english')

combined_caption['caption']=combined_caption['caption'].apply(lambda x: ([word for word in x if word not in (stop)]))

print(combined_caption.head(5))

          video                                     caption
0   video3.webm                [blond, woman, massag, tilt]
1   video4.webm  [roulett, tabl, spin, ball, closeup, shot]
2   video6.webm                             [khr, gangster]
3   video8.webm           [medic, helicopt, hover, airport]
4  video10.webm         [coupl, relax, picnic, crane, shot]


In [17]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(str(e) for e in combined_caption['caption'])


# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
blond,0.572361
tilt,0.564078
massag,0.539078
woman,0.252218
potato,0.000000
...,...
floweri,0.000000
flowerpot,0.000000
flurri,0.000000
flute,0.000000


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(str(e) for e in combined_caption['caption'])
z=x.toarray()


In [45]:
z.shape

(8000, 4458)

**3.Combing a column of vectorized array in caption data frame**

---



In [0]:
combined_caption['tf-idf_verctor']=pd.Series(z.tolist())

In [21]:
combined_caption

,video,caption,tf-idf_verctor
0,video3.webm,"[blond, woman, massag, tilt]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,video4.webm,"[roulett, tabl, spin, ball, closeup, shot]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,video6.webm,"[khr, gangster]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,video8.webm,"[medic, helicopt, hover, airport]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,video10.webm,"[coupl, relax, picnic, crane, shot]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
7995,video10004.webm,"[astronaut, outer, space, backdrop, planet, ea...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7996,video10005.webm,"[young, women, lie, sunb, appli, sun, cream]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7997,video10006.webm,"[doctor, talk, patient, use, tablet, explain, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7998,video10007.webm,"[businessman, sit, beach, inflat, mattress, wo...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Loading C3D feature of Video


**1.Load C3D into DataFrame**

In [0]:
import tqdm
from tqdm import tqdm_notebook as tqdm
from glob import glob
from pathlib import Path

In [24]:
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D
c3d_path = Path('Dev-set/C3D')
max_iterations = 6000
c3d_feat_dict ={}

for file in tqdm (list(c3d_path.glob('*.txt'))):
  key = file.with_suffix('.webm').name
  c3d_feat_dict[key] = read_C3D(file)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


**2.Loading the C3D dictonary into a DataFrame**

---



In [25]:

c3d_train= pd.DataFrame(c3d_feat_dict).T
c3d_train=pd.DataFrame(c3d_train).rename_axis('video')
c3d_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
video,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
video6632.webm,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.700000e-07,0.000000e+00,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000,0.000000e+00,1.331800e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e-07,0.000000e+00,9.800000e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.001623,0.970125,0.000016,0.001298,0.000032,0.000001,0.000000,0.000000e+00,1.000000e-08,2.500000e-07,0.000000e+00,0.000000e+00,5.000000e-08,0.000000e+00,1.000000e-08,1.000000e-08,4.200000e-07,0.000000e+00,3.000000e-08,2.000000e-08,0.000000,0.000000,0.000000e+00,6.000000e-08,0.000000e+00,0.000000e+00,9.000000e-08,0.000000e+00,0.000000e+00,1.100000e-07,1.400000e-07,0.000000e+00,0.000000e+00,1.700000e-07,0.000000e+00,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08
video6634.webm,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,7.339000e-05,3.700000e-06,3.371100e-04,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,0.000001,2.710000e-06,2.300000e-07,1.900000e-07,2.937620e-03,5.200000e-07,9.200000e-07,1.136000e-05,1.055000e-05,3.310400e-04,9.200000e-07,2.189000e-05,8.000000e-08,2.769000e-05,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000045,0.000027,0.000013,0.000077,0.000050,0.000138,0.000012,7.190000e-06,4.100000e-07,5.440000e-06,3.400000e-07,6.870000e-06,8.399000e-05,4.070000e-06,3.970000e-06,5.900000e-07,1.285300e-04,8.800000e-07,2.228000e-05,1.105000e-05,0.000003,0.000015,1.361000e-05,4.090000e-06,3.320000e-06,4.600000e-06,4.920000e-06,5.290000e-06,6.400000e-07,2.372000e-05,6.610000e-06,1.024000e-05,2.030000e-06,5.800000e-06,1.490000e-06,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04
video6633.webm,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,7.750000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.999985,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06
video6645.webm,0.005959,0.004765,0.003757,5.787100e-04,7.344000e-05,1.288200e-04,4.665300e-04,1.026567e-02,3.028100e-04,2.196500e-04,2.521800e-04,2.963730e-03,0.000032,0.000592,3.260000e-05,1.466430e-03,2.430700e-04,0.000181,4.019180e-03,2.390960e-03,1.206900e-04,8.488490e-03,5.216200e-04,4.164600e-04,1.643420e-02,4.968638e-02,1.470550e-03,5.432000e-05,1.055230e-03,5.442770e-03,1.727700e-04,0.024688,9.640700e-04,0.000880,1.140529e-02,4.522160e-03,8.594500e-04,6.573000e-04,6.942400e-04,1.823110e-03,...,0.021621,0.000639,0.014109,0.000472,0.002461,0.004046,0.000905,9.967200e-04,2.514660e-03,1.587440e-03,1.125030e-03,1.008410e-03,2.133877e-02,6.474700e-04,1.373010e-03,9.568000e-05,5.635083e-02,1.62

# Merging caption and c3d features together in a single DataFrame

**1.Combining C3D to Vectorized Caption**

In [26]:
Two_Feature=pd.merge(combined_caption,c3d_train,on='video')
Two_Feature

,video,caption,tf-idf_verctor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,video3.webm,"[blond, woman, massag, tilt]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.020249,0.001578,0.000826,0.000945,6.279000e-05,0.000003,1.161820e-03,0.000097,0.000022,1.033000e-05,0.000337,6.363100e-04,1.111700e-04,0.001008,3.610000e-06,0.000631,3.905000e-05,0.000041,9.125000e-05,0.000303,1.541000e-05,3.197000e-05,5.221000e-05,6.155000e-05,0.017465,6.658100e-04,0.000065,0.000054,2.731800e-04,1.385880e-03,0.000003,0.001356,0.000517,2.426120e-03,2.719140e-03,2.770000e-06,1.557080e-03,...,0.096647,0.004458,0.011840,0.000287,0.002195,0.002792,1.774000e-05,0.003365,6.105000e-05,0.000573,2.725700e-04,0.000034,6.737360e-03,2.083000e-05,0.000098,2.502000e-05,0.005768,2.990000e-06,0.000085,0.000607,0.000461,0.001189,2.702500e-04,0.000127,0.000106,0.000154,5.666400e-04,1.960000e-05,8.938800e-04,0.001079,0.001042,1.613500e-04,2.571600e-04,0.046617,1.561300e-04,6.040000e-06,0.000537,0.000339,0.008437,0.000470
1,video4.webm,"[roulett, tabl, spin, ball, closeup, shot]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000118,0.000891,0.000188,0.000045,6.330000e-05,0.000002,6.406100e-04,0.000011,0.000017,1.396000e-05,0.000336,4.080540e-03,5.900000e-07,0.000411,2.570000e-06,0.000243,6.600000e-07,0.000002,5.352600e-04,0.000278,4.804000e-05,1.152880e-03,2.792000e-05,4.034000e-05,0.022276,4.529320e-03,0.000020,0.000029,4.501000e-05,6.043553e-02,0.000039,0.027208,0.000074,2.719000e-05,6.225100e-04,2.609600e-04,1.834500e-04,...,0.000372,0.000154,0.000148,0.000086,0.000267,0.000014,1.429710e-03,0.004417,6.437000e-05,0.000105,3.128610e-03,0.000055,6.118000e-05,2.513000e-05,0.004283,1.032000e-05,0.000062,2.903000e-05,0.000374,0.000139,0.741702,0.007178,3.456700e-04,0.000104,0.000181,0.000052,4.838610e-03,7.809000e-05,4.232508e-02,0.000623,0.000582,3.925200e-04,8.639800e-04,0.000947,1.355800e-04,6.680000e-06,0.000360,0.000159,0.001025,0.000020
2,video6.webm,"[khr, gangster]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.011765,0.000746,0.000784,0.000013,6.810000e-06,0.000028,4.069000e-05,0.000014,0.000041,8.288000e-05,0.000048,7.962000e-05,1.066610e-03,0.000958,1.143000e-05,0.000070,2.992000e-05,0.000038,3.073000e-05,0.000008,9.700000e-07,2.905050e-03,1.849000e-05,1.347000e-05,0.000192,4.972000e-05,0.000532,0.000027,5.094000e-05,1.000000e-05,0.000005,0.000029,0.000033,3.260172e-02,3.366000e-05,7.100000e-07,6.060000e-06,...,0.876291,0.005509,0.008852,0.000399,0.000069,0.013166,1.459000e-05,0.000044,5.200800e-04,0.000046,7.350000e-06,0.000024,1.049062e-02,4.745400e-04,0.000023,5.590000e-06,0.005554,2.500000e-07,0.000057,0.000328,0.000149,0.001844,8.410000e-06,0.000103,0.000030,0.000005,1.792200e-04,1.110000e-06,5.950000e-06,0.000215,0.000224,2.590000e-06,3.066000e-05,0.002538,1.037400e-04,4.680000e-06,0.000064,0.005380,0.001027,0.001384
3,video8.webm,"[medic, helicopt, hover, airport]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000223,0.000165,0.000007,0.000016,5.340000e-06,0.000014,1.536000e-04,0.000008,0.000004,9.300000e-06,0.001057,1.255400e-04,3.404000e-05,0.000004,5.560000e-06,0.000012,1.940000e-06,0.000003,1.758540e-03,0.000005,8.330000e-06,2.721000e-05,3.200000e-07,1.017000e-05,0.000137,6.290000e-06,0.000005,0.000008,8.400000e-06,6.108600e-04,0.000166,0.583214,0.000004,2.100000e-07,1.570000e-06,2.136000e-05,2.670000e-06,...,0.000009,0.000007,0.000005,0.000539,0.000035,0.000391,2.065000e-05,0.000043,1.017000e-05,0.000028,1.247148e-02,0.006671,1.186780e-03,1.580000e-06,0.119488,5.600000e-07,0.000004,4.250000e-06,0.000040,0.000148,0.103989,0.058989,2.220000e-06,0.000069,0.000004,0.000130,1.182470e-03,2.670000e-06,6.160000e-06,0.000044,0.000046,8.730000e-06,2.278000e-05,0.000053,4.790000e-05,1.950000e-05,0.000001,0.000004,0.000380,0.000029
4,video10.webm,"[c

**bold text**

In [0]:
c3d_grnd=pd.merge(grndtrth_train,c3d_train,on='video')

In [28]:
c3d_grnd

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.924,0.846,0.020249,0.001578,0.000826,0.000945,6.279000e-05,0.000003,1.161820e-03,0.000097,0.000022,1.033000e-05,0.000337,6.363100e-04,1.111700e-04,0.001008,3.610000e-06,0.000631,3.905000e-05,0.000041,9.125000e-05,0.000303,1.541000e-05,3.197000e-05,5.221000e-05,6.155000e-05,0.017465,6.658100e-04,0.000065,0.000054,2.731800e-04,1.385880e-03,0.000003,0.001356,0.000517,2.426120e-03,2.719140e-03,2.770000e-06,1.557080e-03,...,0.096647,0.004458,0.011840,0.000287,0.002195,0.002792,1.774000e-05,0.003365,6.105000e-05,0.000573,2.725700e-04,0.000034,6.737360e-03,2.083000e-05,0.000098,2.502000e-05,0.005768,2.990000e-06,0.000085,0.000607,0.000461,0.001189,2.702500e-04,0.000127,0.000106,0.000154,5.666400e-04,1.960000e-05,8.938800e-04,0.001079,0.001042,1.613500e-04,2.571600e-04,0.046617,1.561300e-04,6.040000e-06,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,0.667,0.000118,0.000891,0.000188,0.000045,6.330000e-05,0.000002,6.406100e-04,0.000011,0.000017,1.396000e-05,0.000336,4.080540e-03,5.900000e-07,0.000411,2.570000e-06,0.000243,6.600000e-07,0.000002,5.352600e-04,0.000278,4.804000e-05,1.152880e-03,2.792000e-05,4.034000e-05,0.022276,4.529320e-03,0.000020,0.000029,4.501000e-05,6.043553e-02,0.000039,0.027208,0.000074,2.719000e-05,6.225100e-04,2.609600e-04,1.834500e-04,...,0.000372,0.000154,0.000148,0.000086,0.000267,0.000014,1.429710e-03,0.004417,6.437000e-05,0.000105,3.128610e-03,0.000055,6.118000e-05,2.513000e-05,0.004283,1.032000e-05,0.000062,2.903000e-05,0.000374,0.000139,0.741702,0.007178,3.456700e-04,0.000104,0.000181,0.000052,4.838610e-03,7.809000e-05,4.232508e-02,0.000623,0.000582,3.925200e-04,8.639800e-04,0.000947,1.355800e-04,6.680000e-06,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,0.700,0.011765,0.000746,0.000784,0.000013,6.810000e-06,0.000028,4.069000e-05,0.000014,0.000041,8.288000e-05,0.000048,7.962000e-05,1.066610e-03,0.000958,1.143000e-05,0.000070,2.992000e-05,0.000038,3.073000e-05,0.000008,9.700000e-07,2.905050e-03,1.849000e-05,1.347000e-05,0.000192,4.972000e-05,0.000532,0.000027,5.094000e-05,1.000000e-05,0.000005,0.000029,0.000033,3.260172e-02,3.366000e-05,7.100000e-07,6.060000e-06,...,0.876291,0.005509,0.008852,0.000399,0.000069,0.013166,1.459000e-05,0.000044,5.200800e-04,0.000046,7.350000e-06,0.000024,1.049062e-02,4.745400e-04,0.000023,5.590000e-06,0.005554,2.500000e-07,0.000057,0.000328,0.000149,0.001844,8.410000e-06,0.000103,0.000030,0.000005,1.792200e-04,1.110000e-06,5.950000e-06,0.000215,0.000224,2.590000e-06,3.066000e-05,0.002538,1.037400e-04,4.680000e-06,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,0.818,0.000223,0.000165,0.000007,0.000016,5.340000e-06,0.000014,1.536000e-04,0.000008,0.000004,9.300000e-06,0.001057,1.255400e-04,3.404000e-05,0.000004,5.560000e-06,0.000012,1.940000e-06,0.000003,1.758540e-03,0.000005,8.330000e-06,2.721000e-05,3.200000e-07,1.017000e-05,0.000137,6.290000e-06,0.000005,0.000008,8.400000e-06,6.108600e-04,0.000166,0.583214,0.000004,2.100000e-07,1.570000e-06,2.136000e-05,2.670000e-06,...,0.000009,0.000007,0.000005,0.000539,0.000035,0.000391,2.065000e-05,0.000043,1.017000e-05,0.000028,1.247148e-02,0.006671,1.186780e-03,1.580000e-06,0.119488,5.600000e-07,0.000004,4.250000e-06,0.000040,0.000148,0.103989,0.058989,2.220000e-06,0.000069,0.000004,0.000130,1.182470e-03,2.670000e-06,6.160000e-06,0.000044,0.000046,8.730000e-06,2.278000e-05,0.000053,4.790000e-05,1.950000e-05,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,0.900,0.000090,0.000615,0.003436,0.001281,3.551400e-03,0.000313,4.187000e-05,0.000018,0.000157,1.171300e-04,0.000256,1.159400e-04,3.234800e-04,0.009888,1.050000e-06,0.000576,3.004000e-05,0.000008,2.028870e-03,0.000016,7.690000e-06,7.324250e-03,2.863000e-05,1.5000

# Spliting the data into train and test for only caption(tf-idf)

1.Dividing Train and Test

In [0]:
X_train = z[0:4800] #tf-idf vectorized array stored in z
Y_train = grndtrth_train.iloc[0:4800, 1:3].values
X_test = z[4800:6000]
Y_test=grndtrth_train.iloc[4800:6000, 1:3].values

In [35]:
grndtrth_train.iloc[:, 1:3].values

array([[0.924, 0.846],
       [0.923, 0.667],
       [0.863, 0.7  ],
       ...,
       [0.713, 0.818],
       [0.954, 1.   ],
       [0.953, 0.615]])

In [0]:
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2) # 

In [36]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (4800, 4458)
X_test (1200, 4458)
Y_train (4800, 2)
Y_test (1200, 2)


# Model (Caption Only)

In [38]:
from sklearn.feature_extraction.text import *
from sklearn.ensemble import *
from sklearn.model_selection import *
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

**Predicting Short term and Long Term for 1200 Test set**

In [44]:
Y_predicted = RFregressor.predict(X_test)
Y_predicted.shape

(1200, 2)

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

**Calculating Spearmens Score**

In [0]:
# The spearmen score is calculate below in different cell as it takes computaion time to again run the model

****

# Predicting Value of Test -set caption(2000)

In [0]:
X_train = z[0:6000] #tf-idf vectorized array stored in z
Y_train = grndtrth_train.iloc[0:6000, 1:3].values
X_test = z[6000:8000]


In [51]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)


X_train (6000, 4458)
X_test (2000, 4458)
Y_train (6000, 2)


In [52]:
from sklearn.feature_extraction.text import *
from sklearn.ensemble import *
from sklearn.model_selection import *
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [53]:
Y_predicted = RFregressor.predict(X_test)
Y_predicted.shape

(2000, 2)

In [0]:
pred_df=pd.DataFrame(Y_predicted)

In [0]:
pred_df.to_csv('/content/drive/My Drive/predicted_file.csv')

In [43]:
Get_score(Y_predicted,Y_test)    #  score of Random forest regressor of 4800 ,1200 train and test

The Spearman's correlation coefficient is: 0.374
The Spearman's correlation coefficient is: 0.144


In [0]:


from math import sqrt
from sklearn.metrics import mean_squared_error
rmse_val = [] #to store rmse values for different k
for K in range(20):
  K = K+1
error = sqrt(mean_squared_error(Y_test,Y_predicted)) #calculate rmse
rmse_val.append(error) #store rmse values
print('RMSE value for ', error)

# Spliting the data into train and test only for c3d(dataframe)

In [0]:
 Y1_train= c3d_grnd.iloc[0:4800, 1:3].values

In [0]:
 Y1_test= c3d_grnd.iloc[4800:6000, 1:3].values

In [0]:
X1_train=c3d_grnd.iloc[0:4800,3:].values

In [0]:
X1_test=c3d_grnd.iloc[4800:6000,3:].values

In [0]:
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

In [205]:
print('X1_train', X1_train.shape)
print('X1_test', X1_test.shape)
print('Y1_train', Y1_train.shape)
print('Y1_test', Y1_test.shape)

X_train (4800, 101)
X_test (1200, 101)
Y_train (4800, 2)
Y_test (1200, 2)


# Model (C3D )


**1.KNN Regressor**KNN

In [246]:
 from sklearn.neighbors import KNeighborsRegressor
 neigh = KNeighborsRegressor(n_neighbors=2000)
 neigh.fit(X1_train, Y1_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2000, p=2,
                    weights='uniform')

In [247]:
Y1_predicted = neigh.predict(X1_test)
Y1_predicted

array([[0.8629115, 0.7826485],
       [0.860328 , 0.777648 ],
       [0.859219 , 0.775954 ],
       ...,
       [0.8597375, 0.7771065],
       [0.8644985, 0.7849035],
       [0.8601745, 0.7761955]])

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [248]:
Get_score(Y1_predicted,Y1_test)

The Spearman's correlation coefficient is: 0.265
The Spearman's correlation coefficient is: 0.131


In [232]:

from sklearn import neighbors
from math import sqrt
from sklearn.metrics import mean_squared_error
rmse_val = [] #to store rmse values for different k
for K in range(20):
  K = K+1
error = sqrt(mean_squared_error(Y1_test,Y1_predicted)) #calculate rmse
rmse_val.append(error) #store rmse values
print('RMSE value for ', error)

RMSE value for  0.11583343085972958
